# Nahuatl Notebook for the WHP_EarlyNahuatl_Dataset

This notebook processes Nahuatl dictionary data, analyzing HTML tags, repairing malformed tags, and extracting citations and cross-references. This is a merged version of Todd's version and I where there is a SQLite-based data management approach.

## Step 1: Setup and Imports

In [1]:
import os
import re
import csv
import sqlite3
import hashlib
import glob
from pathlib import Path
from datetime import datetime
from collections import defaultdict, Counter
from typing import List, Dict, Tuple, Optional, Union, Any

# Data processing
import pandas as pd
import numpy as np

# Character encoding detection
import chardet
from unidecode import unidecode

# HTML/XML processing
from bs4 import BeautifulSoup
from inscriptis import get_text

In [2]:
# Configuration constants
WORKING_DIR = 'working_files'
DB_PATH = '../../data/sqLiteDb/nahuatl_processing.db'  # Single database for all operations
DEFAULT_ENCODING = 'utf-8-sig'  # This encoding is the encoding that works when outputing to CSV files to encode certain Nahuatl characters properly.
CHECKPOINT_STAGES = ['initial', 'cleaned', 'final']  # Only save these to SQLite

## Step 2: Data Import and Working Copy Creation

In [3]:
class DataLoader:
    """Unified data loader for CSV and SQLite sources"""
    
    def __init__(self, encoding: str = DEFAULT_ENCODING):
        self.encoding = encoding
        
    def detect_encoding(self, filepath: str) -> str:
        """Auto-detect file encoding if needed"""
        with open(filepath, 'rb') as file:
            raw_data = file.read(10000)  # Read first 10KB
            result = chardet.detect(raw_data)
            return result['encoding'] or 'utf-8'
    
    def load_from_csv(self, filepath: str, auto_detect_encoding: bool = False) -> pd.DataFrame:
        """Load data from CSV with encoding protection"""
        
        # Detect encoding if requested
        encoding = self.detect_encoding(filepath) if auto_detect_encoding else self.encoding
        
        print(f"Loading data from CSV: {filepath}")
        print(f"Encoding: {encoding}")
        
        try:
            df = pd.read_csv(
                filepath,
                encoding=encoding,
                na_values=[''],
                keep_default_na=True,
                dtype=str,  # Keep everything as strings initially
                low_memory=False
            )
            
            print(f"Successfully loaded {len(df):,} rows × {len(df.columns)} columns")
            print(f"Columns: {list(df.columns)[:5]}..." if len(df.columns) > 5 else f"   Columns: {list(df.columns)}")
            
            return df
            
        except UnicodeDecodeError as e:
            print(f"Encoding error: {e}")
            print("Attempting with auto-detected encoding...")
            return self.load_from_csv(filepath, auto_detect_encoding=True)
    
    def load_from_sqlite(self, db_path: str, table_name: str) -> pd.DataFrame:
        """Load data from SQLite database"""
        
        print(f"Loading data from SQLite: {db_path}")
        print(f"Table: {table_name}")
        
        with sqlite3.connect(db_path) as conn:
            # Check if table exists
            tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table'", conn)
            if table_name not in tables['name'].values:
                raise ValueError(f"Table '{table_name}' not found in database")
            
            df = pd.read_sql(f"SELECT * FROM [{table_name}]", conn)
            
        print(f"Successfully loaded {len(df):,} rows × {len(df.columns)} columns")
        
        return df
    
    
    def create_working_copy(self, df: pd.DataFrame) -> Tuple[pd.DataFrame, pd.DataFrame]:
        """Create a working copy while preserving the original"""
        original_df = df.copy(deep=True)
        working_df = df.copy(deep=True)
        
        print(f"Created working copy of data")
        
        return original_df, working_df

## Step 3: Intermediate Save Functions

In [4]:
class DataSaver:
    """Handles saving data at various stages with encoding protection"""
    
    def __init__(self, working_dir: str = WORKING_DIR, encoding: str = DEFAULT_ENCODING):
        self.working_dir = working_dir
        self.encoding = encoding
        os.makedirs(working_dir, exist_ok=True)
        
    def save_to_csv(self, df: pd.DataFrame, filename: str, add_timestamp: bool = True) -> str:
        """Save DataFrame to CSV with encoding protection"""
        
        df_copy = df.copy()
    
        # Headers that need formula escaping - ONLY these two columns
        headers_to_escape = ['Headword', 'Orthographic Variants']
        
        # Add single quote prefix ONLY for specified headers to prevent Excel formula interpretation
        for header in headers_to_escape:
            if header in df_copy.columns:
                df_copy[header] = df_copy[header].astype(str).apply(
                    lambda x: f"'{x}"
                )
            
        if add_timestamp:
            timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
            base_name = filename.replace('.csv', '')
            filename = f"{base_name}_{timestamp}.csv"
        
        filepath = os.path.join(self.working_dir, filename)
        
        df.to_csv(
            filepath,
            index=False,
            encoding=self.encoding,
            na_rep='',  # Explicit NA representation
            quoting=csv.QUOTE_MINIMAL
        )
        
        print(f"Saved to CSV: {filepath}")
        return filepath
    
    def save_checkpoint_to_sqlite(self, df: pd.DataFrame, checkpoint_name: str, conn: sqlite3.Connection) -> None:
        """Save checkpoint to SQLite (only for critical stages)"""
        
        table_name = f"checkpoint_{checkpoint_name}_{datetime.now().strftime('%Y%m%d')}"
        
        df.to_sql(
            table_name,
            conn,
            if_exists='replace',
            index=False,
            dtype='text'  # Store everything as text to preserve formatting
        )
        
        print(f"Checkpoint saved to SQLite: {table_name}")
        
        # Also save metadata
        metadata = pd.DataFrame([{
            'checkpoint_name': checkpoint_name,
            'table_name': table_name,
            'timestamp': datetime.now().isoformat(),
            'row_count': len(df),
            'column_count': len(df.columns),
            'columns': ','.join(df.columns)
        }])
        
        metadata.to_sql(
            'checkpoint_metadata',
            conn,
            if_exists='append',
            index=False
        )
    
    def save_to_excel(self, data_dict: Dict[str, pd.DataFrame], filename: str) -> str:
        """Save multiple DataFrames to Excel file"""
        
        filepath = os.path.join(self.working_dir, filename)
        
        with pd.ExcelWriter(filepath, engine='openpyxl') as writer:
            for sheet_name, df in data_dict.items():
                # Excel sheet name limit is 31 characters
                clean_sheet_name = sheet_name[:31] if len(sheet_name) > 31 else sheet_name
                df.to_excel(writer, sheet_name=clean_sheet_name, index=False)
        
        print(f"Saved to Excel: {filepath}")
        return filepath

## Step 4: Database Initialization and Setup

In [5]:
class DatabaseManager:
    """Manages SQLite database connection and operations"""
    
    def __init__(self, db_path: str = DB_PATH):
        self.db_path = db_path
        self.conn = None
        self._initialize_database()
    
    def _initialize_database(self):
        """Initialize database and create necessary tables"""
        
        # Create directory if it doesn't exist
        db_dir = os.path.dirname(self.db_path)
        if db_dir:
            os.makedirs(db_dir, exist_ok=True)
        
        # Connect to database
        self.conn = sqlite3.connect(self.db_path)
        
        print(f"Database initialized: {self.db_path}")
        
        # Create metadata tables if they don't exist
        self._create_metadata_tables()
        
        # Show existing tables
        self._show_database_info()
    
    def _create_metadata_tables(self):
        """Create metadata tables for tracking processing"""
        
        # Checkpoint metadata table
        if self.conn is None:
            raise ValueError("Database connection is not initialized.")
        
        self.conn.execute("""
            CREATE TABLE IF NOT EXISTS checkpoint_metadata (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            checkpoint_name TEXT,
            table_name TEXT,
            timestamp TEXT,
            row_count INTEGER,
            column_count INTEGER,
            columns TEXT
            )
        """)
        
        # Processing log table
        self.conn.execute("""
            CREATE TABLE IF NOT EXISTS processing_log (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                stage TEXT,
                status TEXT,
                timestamp TEXT,
                duration_seconds REAL,
                rows_processed INTEGER,
                notes TEXT
            )
        """)
        
        self.conn.commit()
    
    
    
    def import_initial_dataset(self, csv_path: str, table_name: str = 'WHP_EarlyNahuatl_Data', 
                              encoding: str = DEFAULT_ENCODING, replace: bool = False) -> bool:
        """
        Import initial CSV dataset into SQLite as the base reference.
        This is a ONE-TIME operation to establish the source data in the database.
        """
        
        print("\n" + "=" * 70)
        print("INITIAL DATASET IMPORT TO SQLITE")
        print("=" * 70)
        
        # Check if table already exists
        existing_tables = pd.read_sql(
            "SELECT name FROM sqlite_master WHERE type='table'",
            self.conn
        )
        
        if table_name in existing_tables['name'].values:
            if not replace:
                print(f"Table '{table_name}' already exists!")
                response = input("Do you want to replace it? (yes/no): ").lower()
                if response != 'yes':
                    print("Import cancelled")
                    return False
            else:
                print(f" Replacing existing table '{table_name}'")
        
        try:
            # Load CSV with encoding protection
            print(f"Reading CSV file: {csv_path}")
            print(f"   Encoding: {encoding}")
            
            # First, detect actual encoding if needed
            with open(csv_path, 'rb') as file:
                raw_data = file.read(10000)
                detected = chardet.detect(raw_data)
                print(f"   Detected encoding: {detected['encoding']} (confidence: {detected['confidence']:.2%})")
            
            # Load the CSV
            df = pd.read_csv(
                csv_path,
                encoding=encoding,
                na_values=[''],
                keep_default_na=True,
                dtype=str,  # Import everything as text to preserve formatting
                low_memory=False
            )
            
            print(f"Loaded {len(df):,} rows × {len(df.columns)} columns")
            
            # Show sample of data
            print("\nData Sample (first 3 rows):")
            print(df.head(3).to_string(max_cols=5))
            
            # Import to SQLite
            print(f"\nImporting to SQLite table: {table_name}")
            
            df.to_sql(
                table_name,
                self.conn,
                if_exists='replace' if replace else 'fail',
                index=False,
                dtype='text',  # Store as text to preserve all characters
                chunksize=5000  # Process in chunks for large datasets
            )
            
            # Create indexes for common query columns
            print("Creating indexes...")
            
            if self.conn is None:
                raise ValueError("Database connection is not initialized.")
        
            # Add index on Ref column if it exists
            if 'Ref' in df.columns:
                self.conn.execute(f"CREATE INDEX IF NOT EXISTS idx_{table_name}_ref ON {table_name}(Ref)")
            
            # Add index on Headword column if it exists  
            if 'Headword' in df.columns:
                self.conn.execute(f"CREATE INDEX IF NOT EXISTS idx_{table_name}_headword ON {table_name}(Headword)")
            
            self.conn.commit()
            
            # Verify import
            verify_count = pd.read_sql(f"SELECT COUNT(*) as cnt FROM {table_name}", self.conn).iloc[0, 0]
            
            if verify_count == len(df):
                print(f"Successfully imported {verify_count:,} rows")
                
                # Log the import
                import_metadata = pd.DataFrame([{
                    'operation': 'initial_import',
                    'source_file': csv_path,
                    'table_name': table_name,
                    'timestamp': datetime.now().isoformat(),
                    'row_count': verify_count,
                    'column_count': len(df.columns),
                    'columns': ','.join(df.columns),
                    'encoding_used': encoding
                }])
                
                import_metadata.to_sql(
                    'import_history',
                    self.conn,
                    if_exists='append',
                    index=False
                )
                
                print("Import logged to metadata")
                return True
            else:
                print(f"Row count mismatch! Expected {len(df)}, got {verify_count}")
                return False
                
        except Exception as e:
            print(f"Import failed: {e}")
            return False
        
        
    def _show_database_info(self):
        """Display information about the database"""
        
        tables = pd.read_sql(
            "SELECT name, type FROM sqlite_master WHERE type IN ('table', 'view')",
            self.conn
        )
        
        if not tables.empty:
            print(f"Existing database objects: {len(tables)}")
            for _, row in tables.iterrows():
                # Get row count for tables
                if row['type'] == 'table':
                    try:
                        count = pd.read_sql(f"SELECT COUNT(*) as cnt FROM [{row['name']}]", self.conn).iloc[0, 0]
                        print(f"   - {row['name']} ({row['type']}): {count:,} rows")
                    except:
                        print(f"   - {row['name']} ({row['type']})")
        else:
            print("No existing tables in database")
    
    def log_processing_stage(self, stage: str, status: str, 
                        duration: Optional[float] = None, 
                        rows_processed: Optional[int] = None, 
                        notes: Optional[str] = None):
        """Log processing stage to database"""
        
        log_entry = pd.DataFrame([{
            'stage': stage,
            'status': status,
            'timestamp': datetime.now().isoformat(),
            'duration_seconds': duration,
            'rows_processed': rows_processed,
            'notes': notes
        }])
        
        log_entry.to_sql('processing_log', self.conn, if_exists='append', index=False)
    
    
    def verify_base_dataset(self, table_name: str = 'WHP_EarlyNahuatl_Data') -> bool:
        """Verify that base dataset exists and is valid"""
        
        try:
            # Check if table exists
            tables = pd.read_sql(
                f"SELECT name FROM sqlite_master WHERE type='table' AND name='{table_name}'",
                self.conn
            )
            
            if tables.empty:
                print(f"Base dataset table '{table_name}' not found")
                return False
            
            # Get basic info
            count = pd.read_sql(f"SELECT COUNT(*) as cnt FROM {table_name}", self.conn).iloc[0, 0]
            cols = pd.read_sql(f"PRAGMA table_info({table_name})", self.conn)
            
            print(f"   Base dataset verified:")
            print(f"   Table: {table_name}")
            print(f"   Rows: {count:,}")
            print(f"   Columns: {len(cols)}")
            
            return True
            
        except Exception as e:
            print(f"Verification failed: {e}")
            return False
        
        
    def get_checkpoint_list(self) -> pd.DataFrame:
        """Get list of available checkpoints"""
        
        try:
            checkpoints = pd.read_sql(
                "SELECT * FROM checkpoint_metadata ORDER BY timestamp DESC",
                self.conn
            )
            return checkpoints
        except:
            return pd.DataFrame()
    
    def load_checkpoint(self, checkpoint_name: Optional[str] = None, 
                   table_name: Optional[str] = None) -> pd.DataFrame:
        """Load a specific checkpoint"""
        
        if table_name:
            return pd.read_sql(f"SELECT * FROM [{table_name}]", self.conn)
        elif checkpoint_name:
            # Get most recent checkpoint with this name
            query = """
                SELECT table_name 
                FROM checkpoint_metadata 
                WHERE checkpoint_name = ? 
                ORDER BY timestamp DESC 
                LIMIT 1
            """
            result = pd.read_sql(query, self.conn, params=[checkpoint_name])
            if not result.empty:
                table_name = str(result.iloc[0, 0])
                return pd.read_sql(f"SELECT * FROM [{table_name}]", self.conn)
        
        raise ValueError("No checkpoint found")
    
    def close(self):
        """Close database connection"""
        if self.conn:
            self.conn.close()
            print("Database connection closed")

## Step 5: HTML Tag Handler (Detection + Repair)

In [6]:
class HTMLTagHandler:
    """Handles HTML tag detection and repair"""
    
    def __init__(self):
        # Valid HTML tags
        self.valid_tags = {
            'p', 'br', 'div', 'span', 'a', 'b', 'i', 'u', 'strong', 'em',
            'h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'ul', 'ol', 'li', 'table',
            'tr', 'td', 'th', 'img', 'link', 'meta', 'head', 'body', 'html',
            'bibl', 'title', 'sup', 'sub', 'del'
        }
        
        # Known malformations and their fixes
        self.malformed_fixes = {
            '</p</bibl>': '</p></bibl>',       
            '<bibl<': '<bibl>',
            '</bibbl>': '</bibl>',
            '<bibbl>': '<bibl>',
            '<bobl>': '<bibl>',
            '</bobl>': '</bibl>',
            '<b9bl>': '<bibl>',
            '<bibi>': '<bibl>',
            '<bibl></p>': '</bibl></p>',
        }
    
    def find_all_tags(self, text: str) -> List[str]:
        """Find all angle bracket patterns"""
        if pd.isna(text) or not text:
            return []
        return re.findall(r'</?[^<>]+/?>', str(text))
    
    def classify_tag(self, tag: str) -> Tuple[str, Optional[str]]:
        """
        Classify a tag as: valid_html, malformed_html, or non_html
        Returns: (tag_type, suggested_fix)
        """
        # Check if it's a known malformation
        if tag in self.malformed_fixes:
            return 'malformed_html', self.malformed_fixes[tag]
        
        # Extract tag name
        match = re.match(r'^</?([^>\s/]+)', tag)
        if match:
            tag_name = match.group(1).lower()
            if tag_name in self.valid_tags:
                return 'valid_html', None
            
            # Check if it looks like HTML but misspelled
            if any(valid in tag_name for valid in ['bibl', 'p', 'br', 'div']):
                # Suggest a fix based on what it looks like
                if 'bibl' in tag_name:
                    return 'malformed_html', '</bibl>' if tag.startswith('</') else '<bibl>'
                return 'malformed_html', tag  # Can't auto-fix
        
        # Not HTML at all
        return 'non_html', None
    
    def repair_text(self, text: str) -> Tuple[str, int]:
        """
        Repair all malformed HTML tags in text
        Returns: (repaired_text, number_of_repairs)
        """
        if pd.isna(text) or not text:
            return text, 0
        
        text_str = str(text)
        repairs_made = 0
        
        # Apply known fixes
        for malformed, fix in self.malformed_fixes.items():
            if malformed in text_str:
                count = text_str.count(malformed)
                text_str = text_str.replace(malformed, fix)
                repairs_made += count
        
        # Apply pattern-based fixes
        text_str = re.sub(r'<<(\w+)>', r'<\1>', text_str)  # Double opening
        text_str = re.sub(r'<(\w+)>>', r'<\1>', text_str)  # Double closing
        
        return text_str, repairs_made
    
    def encode_non_html(self, text: str) -> str:
        """Encode non-HTML angle brackets"""
        if pd.isna(text) or not text:
            return text
        
        text_str = str(text)
        tags = self.find_all_tags(text_str)
        
        for tag in tags:
            tag_type, _ = self.classify_tag(tag)
            if tag_type == 'non_html':
                encoded = tag.replace('<', '&lt;').replace('>', '&gt;')
                text_str = text_str.replace(tag, encoded)
        
        return text_str

### Step 5.1 HTML Malformed Tag Repair

In [7]:
class MalformedTagRepairer:
    """Specialized handler for malformed HTML tag repair"""
    
    def __init__(self):
        self.html_analyzer = HTMLTagHandler() 
        self.html_tags = {
            'p', 'br', 'div', 'span', 'a', 'b', 'i', 'u', 'strong', 'em',
            'h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'ul', 'ol', 'li', 'table',
            'tr', 'td', 'th', 'img', 'bibl', 'title', 'sup', 'sub', 'del'
        }
        self.self_closing_tags = {'br', 'hr', 'img', 'input', 'meta', 'link'}
        # Specific patterns for actual HTML malformations
        self.html_malformation_patterns = {
            '</bibbl>': '</bibl>',
            '<bibbl>': '<bibl>',
            '</bobl>': '</bibl>',
            '<bobl>': '<bibl>',
            '<b9bl>': '<bibl>',
            '<bibi>': '<bibl>',
            '</p</bibl>': '</p></bibl>',
            '<p<': '<p>',
            '</p>p>': '</p>',
        }
        self.auto_repairable_tags = {'bibl'}    
        self.tag_closing_heuristics = {
            'bibl': [
                r'</p>',           # End at paragraph close
                r'\n\s*<p',        # Before new paragraph  
                r'\.\s*(?=<|$)',   # After period at end of sentence
                r'$'               # End of text as fallback
            ]
        }
        self.regex_patterns = {
            r'<<(\w+)>': r'<\1>',        # Double opening
            r'<(\w+)>>': r'<\1>',        # Double closing
        }
        
    def detect_mismatches(self, text: str) -> list[dict[str, str]]:
        """Detect tag count mismatches"""
        if pd.isna(text) or text == '':
            return []
        
        mismatches = []
        text_str = str(text)
        
        for tag_name in self.html_tags:
            if tag_name in self.self_closing_tags:
                continue
            
            # Count opening and closing tags
            open_pattern = f"<{tag_name}(?:\\s+[^>]*)?>"
            close_pattern = f'</{tag_name}>'
            
            open_count = len(re.findall(open_pattern, text_str, re.IGNORECASE))
            close_count = len(re.findall(close_pattern, text_str, re.IGNORECASE))
            
            if open_count != close_count:
                mismatches.append({
                    'tag': tag_name,
                    'open_count': open_count,
                    'close_count': close_count,
                    'issue': f'Mismatch: {open_count} open, {close_count} closed'
                })
        
        return mismatches
    
    
    def analyze_malformed_tags(self, df: pd.DataFrame) -> Dict[str, pd.DataFrame]:
        """Analyze both malformed patterns AND mismatches"""
        from collections import Counter
        
        results = {
            'malformed_by_row': [],
            'malformed_summary': []
        }
        
        for idx, row in df.iterrows():
            for col in df.columns:
                cell_value = row[col]
                if pd.notna(cell_value) and cell_value != '':
                    text_str = str(cell_value)
                    
                    # Check for literal malformed patterns
                    for pattern, replacement in self.html_malformation_patterns.items():
                        if pattern in text_str:
                            results['malformed_by_row'].append({
                                'Row': idx,
                                'Column': col,
                                'Malformed_Tag': pattern,
                                'Suggested_Repair': replacement,
                                'Context': text_str[:100]
                            })
                    
                    # ADD MISMATCH DETECTION
                    mismatches = self.detect_mismatches(text_str)
                    for mismatch in mismatches:
                        results['malformed_by_row'].append({
                            'Row': idx,
                            'Column': col,
                            'Malformed_Tag': f"<{mismatch['tag']}>",
                            'Suggested_Repair': mismatch['issue'],
                            'Context': text_str[:100]
                        })
                        
        for idx, row in df.iterrows():
            for col in df.columns:
                cell_value = row[col]
                if pd.notna(cell_value) and cell_value != '':
                    text_str = str(cell_value)
                    
                    # Check regex patterns
                    for pattern, replacement in self.regex_patterns.items():
                        if re.search(pattern, text_str):
                            results['malformed_by_row'].append({
                                'Row': idx,
                                'Column': col,
                                'Malformed_Tag': f"REGEX:{pattern}",
                                'Suggested_Repair': replacement,
                                'Context': text_str[:100]
                            })
        # Create summary
        if results['malformed_by_row']:
            pattern_counts = Counter(item['Malformed_Tag'] for item in results['malformed_by_row'])
            
            for pattern, count in pattern_counts.items():
                first_occurrence = next(item for item in results['malformed_by_row'] 
                                       if item['Malformed_Tag'] == pattern)
                
                results['malformed_summary'].append({
                    'Malformed_Tag': pattern,
                    'Count': count,
                    'Suggested_Repair': first_occurrence['Suggested_Repair'],
                    'Sample_Context': first_occurrence['Context']
                })
        
        malformed_by_row_df = pd.DataFrame(results['malformed_by_row'])
        malformed_summary_df = pd.DataFrame(results['malformed_summary'])
        
        if not malformed_summary_df.empty:
            malformed_summary_df = malformed_summary_df.sort_values('Count', ascending=False)
        
        return {
            'Malformed_Tags_by_Row': malformed_by_row_df,
            'Malformed_Tags_Summary': malformed_summary_df
        }
    
    def repair_tags(self, df: pd.DataFrame, malformed_analysis: Dict) -> pd.DataFrame:
        """Apply both malformed tag repairs AND mismatch repairs"""
        df_repaired = df.copy()
        
        if malformed_analysis['Malformed_Tags_Summary'].empty:
            return df_repaired
        
        # 1. Apply literal pattern replacements (existing logic)
        literal_repairs = 0
        for _, row in malformed_analysis['Malformed_Tags_Summary'].iterrows():
            pattern = row['Malformed_Tag']
            repair = row['Suggested_Repair']
            
            # Skip mismatches and regex patterns for now
            if "Mismatch:" in str(repair) or pattern.startswith("REGEX:"):
                continue
            
            # Apply literal pattern replacements
            if pattern in self.html_malformation_patterns:
                for col in df_repaired.columns:
                    if df_repaired[col].dtype == 'object':
                        before_count = df_repaired[col].astype(str).str.contains(
                            re.escape(pattern), regex=True
                        ).sum()
                        
                        df_repaired[col] = df_repaired[col].astype(str).str.replace(
                            pattern, repair, regex=False
                        )
                        
                        after_count = df_repaired[col].astype(str).str.contains(
                            re.escape(pattern), regex=True
                        ).sum()
                        
                        literal_repairs += (before_count - after_count)
        
        print(f"  Applied {literal_repairs} literal tag repairs")
        
        # 2. Apply mismatch repairs (NEW!)
        df_repaired = self.repair_tag_mismatches(df_repaired, malformed_analysis)
        
        return df_repaired

    def generate_mismatch_report(self, malformed_analysis: Dict) -> pd.DataFrame:
            """Generate a report of mismatches for manual review"""
            mismatch_rows = []
            
            for _, row in malformed_analysis['Malformed_Tags_Summary'].iterrows():
                if "Mismatch:" in str(row['Suggested_Repair']):
                    mismatch_rows.append({
                        'Tag': row['Malformed_Tag'],
                        'Issue': row['Suggested_Repair'],
                        'Occurrences': row['Count'],
                        'Sample_Context': row['Sample_Context'],
                        'Action_Required': 'Manual review needed'
                    })
            
            return pd.DataFrame(mismatch_rows)
    
    def _extract_mismatch_info(self, malformed_analysis: Dict) -> Dict:
        """Extract mismatch information from analysis"""
        mismatches = {}
        
        for _, row in malformed_analysis['Malformed_Tags_Summary'].iterrows():
            if "Mismatch:" in str(row['Suggested_Repair']):
                tag = row['Malformed_Tag'].replace('<', '').replace('>', '')
                issue = row['Suggested_Repair']
                
                # Parse "Mismatch: X open, Y closed"
                import re
                match = re.search(r'Mismatch: (\d+) open, (\d+) closed', issue)
                if match:
                    mismatches[tag] = {
                        'open_count': int(match.group(1)),
                        'close_count': int(match.group(2)),
                        'occurrences': row['Count']
                    }
        
        return mismatches
    
    def _add_missing_closing_tags(self, df: pd.DataFrame, tag_name: str, missing_count: int) -> tuple[pd.DataFrame, int]:
        """Add missing closing tags at logical boundaries"""
        df_repaired = df.copy()
        total_repairs = 0
        
        for col in df_repaired.columns:
            if df_repaired[col].dtype != 'object':
                continue
                
            for idx in df_repaired.index:
                text = df_repaired.at[idx, col]
                if pd.isna(text) or text == '':
                    continue
                
                text_str = str(text)
                # Check if this cell has the tag mismatch
                open_count = len(re.findall(f"<{tag_name}(?:\\s+[^>]*)?>" , text_str, re.IGNORECASE))
                close_count = len(re.findall(f'</{tag_name}>', text_str, re.IGNORECASE))
                
                if open_count > close_count:
                    cell_missing = open_count - close_count
                    repaired_text = self._repair_text_missing_closes(text_str, tag_name, cell_missing)
                    
                    if repaired_text != text_str:
                        df_repaired.at[idx, col] = repaired_text
                        total_repairs += cell_missing
        
        return df_repaired, total_repairs
    
    def _repair_text_missing_closes(self, text: str, tag_name: str, missing_count: int) -> str:
        """Apply smart heuristics to add missing closing tags"""
        if tag_name not in self.tag_closing_heuristics:
            return text
        
        rules = self.tag_closing_heuristics[tag_name]
        repaired_text = text
        remaining_to_fix = missing_count
        
        # Try each heuristic rule in order of preference
        for rule in rules:
            if remaining_to_fix <= 0:
                break
                
            # Find all potential closing positions
            matches = list(re.finditer(rule, repaired_text))
            
            # Insert closing tags (in reverse order to preserve positions)
            insert_count = min(len(matches), remaining_to_fix)
            for match in reversed(matches[:insert_count]):
                pos = match.start()
                repaired_text = (repaired_text[:pos] + 
                               f'</{tag_name}>' + 
                               repaired_text[pos:])
                remaining_to_fix -= 1
        
        return repaired_text
    
    def _add_missing_opening_tags(self, text: str, tag_name: str, missing_count: int) -> str:
        """Add opening tags before orphaned closing tags using smart heuristics"""
        
        # Define backward-looking heuristics for different tag types
        opening_heuristics = {
            'bibl': [
                r'([^<>\n]{1,100})(\s*</bibl>)', # Longer content for bibliography
            ],
        }
        
        if tag_name not in opening_heuristics:
            return text
        
        repaired_text = text
        rules = opening_heuristics[tag_name]
        remaining_to_fix = missing_count
        
        # Apply heuristics to add opening tags
        for rule in rules:
            if remaining_to_fix <= 0:
                break
                
            # Find matches and add opening tags (process in reverse to preserve positions)
            matches = list(re.finditer(rule, repaired_text))
            for match in reversed(matches[:remaining_to_fix]):
                content = match.group(1)
                closing_part = match.group(2)
                
                # Insert opening tag before the content
                replacement = f'<{tag_name}>{content}{closing_part}'
                start, end = match.span()
                repaired_text = repaired_text[:start] + replacement + repaired_text[end:]
                remaining_to_fix -= 1
        
        return repaired_text
    
    def _fix_orphaned_closing_tags(self, df: pd.DataFrame, tag_name: str, orphaned_count: int) -> tuple[pd.DataFrame, int]:
        """Add missing opening tags for orphaned closing tags"""
        df_repaired = df.copy()
        total_additions = 0
        
        for col in df_repaired.columns:
            if df_repaired[col].dtype != 'object':
                continue
                
            for idx in df_repaired.index:
                text = df_repaired.at[idx, col]
                if pd.isna(text) or text == '':
                    continue
                
                text_str = str(text)
                open_count = len(re.findall(f"<{tag_name}(?:\\s+[^>]*)?>" , text_str, re.IGNORECASE))
                close_count = len(re.findall(f'</{tag_name}>', text_str, re.IGNORECASE))
                
                if close_count > open_count:
                    excess_closes = close_count - open_count
                    repaired_text = self._add_missing_opening_tags(text_str, tag_name, excess_closes)
                    
                    if repaired_text != text_str:
                        df_repaired.at[idx, col] = repaired_text
                        total_additions += excess_closes
        
        return df_repaired, total_additions
    
    def repair_tag_mismatches(self, df: pd.DataFrame, malformed_analysis: Dict) -> pd.DataFrame:
        """Repair ONLY auto-repairable tag mismatches (currently just bibl)"""
        df_repaired = df.copy()
        repairs_made = 0
        manual_review_needed = []
        
        # Extract mismatch information from analysis
        mismatch_info = self._extract_mismatch_info(malformed_analysis)
        
        for tag_name, details in mismatch_info.items():
            open_count = details['open_count']
            close_count = details['close_count']
            
            if tag_name in self.auto_repairable_tags:
                # AUTO-REPAIR: Only bibl tags
                if open_count > close_count:
                    missing_closes = open_count - close_count
                    df_repaired, fixed_count = self._add_missing_closing_tags(
                        df_repaired, tag_name, missing_closes
                    )
                    repairs_made += fixed_count
                    print(f"  ✓ AUTO-FIXED: Added {fixed_count} missing </{tag_name}> tags")
                    
                elif close_count > open_count:
                    orphaned_closes = close_count - open_count
                    df_repaired, added_count = self._fix_orphaned_closing_tags(
                        df_repaired, tag_name, orphaned_closes
                    )
                    repairs_made += added_count
                    print(f"  ✓ AUTO-FIXED: Added {added_count} missing <{tag_name}> opening tags")
            else:
                # MANUAL REVIEW: All other tags
                manual_review_needed.append({
                    'tag': tag_name,
                    'open_count': open_count,
                    'close_count': close_count,
                    'occurrences': details['occurrences']
                })
                print(f"MANUAL REVIEW: <{tag_name}> has {open_count} open, {close_count} closed ({details['occurrences']} occurrences)")
        
        if manual_review_needed:
            print(f"\nSUMMARY: {len(manual_review_needed)} tag types need manual review:")
            for item in manual_review_needed:
                print(f"   - <{item['tag']}>: {item['open_count']} open, {item['close_count']} closed")
        
        print(f"Total automatic repairs: {repairs_made}")
        return df_repaired
        
    def find_mismatch_cells(self, df: pd.DataFrame, target_tags: list) -> pd.DataFrame:
        """Find specific cells that contain tag mismatches"""
        problem_cells = []
        
        for col in df.columns:
            if df[col].dtype != 'object':
                continue
                
            for idx in df.index:
                text = df.at[idx, col]
                if pd.isna(text) or text == '':
                    continue
                    
                text_str = str(text)
                
                for tag_name in target_tags:
                    if tag_name in self.self_closing_tags:
                        continue
                        
                    # Count tags
                    open_count = len(re.findall(f"<{tag_name}(?:\\s+[^>]*)?>" , text_str, re.IGNORECASE))
                    close_count = len(re.findall(f'</{tag_name}>', text_str, re.IGNORECASE))
                    
                    if open_count != close_count:
                        problem_cells.append({
                            'Row': idx + 1,  # +1 for Excel-style numbering
                            'Column': col,
                            'Tag': tag_name,
                            'Open_Count': open_count,
                            'Close_Count': close_count,
                            'Content_Preview': text_str[:200] + '...' if len(text_str) > 200 else text_str
                        })
        
        return pd.DataFrame(problem_cells).sort_values(['Tag', 'Row'])  
    
    def get_repair_statistics(self, malformed_analysis: Dict) -> Dict:
            """Enhanced statistics showing auto vs manual repair breakdown"""
            stats = {
                'total_issues': len(malformed_analysis['Malformed_Tags_by_Row']),
                'unique_patterns': len(malformed_analysis['Malformed_Tags_Summary']),
                'auto_fixable': 0,
                'manual_review_needed': 0,
                'literal_malformations': 0,
                'tag_mismatches': 0,
                'bibl_mismatches': 0,  # Track bibl specifically
                'other_mismatches': 0
            }
            
            for _, row in malformed_analysis['Malformed_Tags_Summary'].iterrows():
                if "Mismatch:" in str(row['Suggested_Repair']):
                    stats['tag_mismatches'] += 1
                    tag_name = row['Malformed_Tag'].replace('<', '').replace('>', '')
                    
                    if tag_name in self.auto_repairable_tags:
                        stats['auto_fixable'] += row['Count']
                        stats['bibl_mismatches'] += row['Count']
                    else:
                        stats['manual_review_needed'] += row['Count'] 
                        stats['other_mismatches'] += row['Count']
                else:
                    stats['auto_fixable'] += row['Count']
                    stats['literal_malformations'] += 1
            
            return stats

    def generate_manual_review_report(self, malformed_analysis: Dict) -> pd.DataFrame:
        """Generate a focused report for manual review (non-bibl tags)"""
        manual_rows = []
        
        for _, row in malformed_analysis['Malformed_Tags_Summary'].iterrows():
            if "Mismatch:" in str(row['Suggested_Repair']):
                tag_name = row['Malformed_Tag'].replace('<', '').replace('>', '')
                
                if tag_name not in self.auto_repairable_tags:
                    manual_rows.append({
                        'Tag': row['Malformed_Tag'],
                        'Issue': row['Suggested_Repair'],
                        'Occurrences': row['Count'],
                        'Sample_Context': row['Sample_Context'],
                        'Action_Required': 'Manual review needed',
                        'Issue_Type': 'Tag Mismatch'
                    })
        
        return pd.DataFrame(manual_rows)

### Step 5.2 NonHTMLTagProcessor

In [8]:
class NonHTMLTagProcessor:
    """Handles encoding of non-HTML angle brackets - OPTIMIZED VERSION"""
    
    def __init__(self):
        self.html_analyzer = HTMLTagHandler()
        
        # Known HTML tags that should never be encoded
        self.valid_html_tags = {
            'p', 'br', 'div', 'span', 'a', 'b', 'i', 'u', 'strong', 'em',
            'h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'ul', 'ol', 'li', 'table',
            'tr', 'td', 'th', 'img', 'bibl', 'title', 'sup', 'sub', 'del'
        }
    
    def find_non_html_tags(self, text: str) -> List[str]:
        """Find tags that are definitely not HTML"""
        if pd.isna(text) or text == '':
            return []
        
        # Find all < ... > patterns
        pattern = r'<[^<>]*>'
        all_brackets = re.findall(pattern, str(text))
        
        non_html_tags = []
        for tag in all_brackets:
            # Check if it's valid HTML
            tag_type, _ = self.html_analyzer.classify_tag(tag)
            if tag_type == 'non_html':
                non_html_tags.append(tag)
        
        return non_html_tags
    
    def analyze_non_html_tags(self, df: pd.DataFrame) -> Dict[str, pd.DataFrame]:
        """OPTIMIZED: Analyze non-HTML tags across DataFrame efficiently"""
        results = {
            'non_html_by_row': [],
            'non_html_summary': []
        }
        
        # Collect all unique non-HTML tags first
        unique_non_html_tags = set()
        tag_locations = defaultdict(list)
        
        # Process in chunks for better memory efficiency
        for idx, row in df.iterrows():
            # Process multiple columns at once
            for col in df.columns:
                cell_value = row[col]
                if pd.notna(cell_value) and cell_value != '':
                    non_html_tags = self.find_non_html_tags(cell_value)
                    for tag in non_html_tags:
                        unique_non_html_tags.add(tag)
                        tag_locations[tag].append({
                            'Row': idx,
                            'Column': col,
                            'Context': str(cell_value)[:100]
                        })
        
        # Build results from collected data
        for tag, locations in tag_locations.items():
            for loc in locations:
                results['non_html_by_row'].append({
                    'Row': loc['Row'],
                    'Column': loc['Column'],
                    'Non_HTML_Tag': tag,
                    'Context': loc['Context']
                })
        
        # Create summary
        if unique_non_html_tags:
            summary_data = []
            for tag in unique_non_html_tags:
                summary_data.append({
                    'Tag': tag,
                    'Count': len(tag_locations[tag]),
                    'Encoded_Version': tag.replace('<', '&lt;').replace('>', '&gt;')
                })
            results['non_html_summary'] = summary_data
        
        non_html_by_row_df = pd.DataFrame(results['non_html_by_row'])
        non_html_summary_df = pd.DataFrame(results['non_html_summary'])
        
        return {
            'Non_HTML_Tags_by_Row': non_html_by_row_df,
            'Non_HTML_Tags_Summary': non_html_summary_df
        }
    
    def encode_brackets(self, df: pd.DataFrame, non_html_analysis: Dict) -> pd.DataFrame:
        """OPTIMIZED: Apply non-HTML bracket encoding using vectorized operations"""
        df_encoded = df.copy()
        
        # Get unique non-HTML tags from analysis
        if not non_html_analysis['Non_HTML_Tags_Summary'].empty:
            # Process all replacements at once using vectorized operations
            for _, row in non_html_analysis['Non_HTML_Tags_Summary'].iterrows():
                tag = row['Tag']
                encoded_tag = row['Encoded_Version']
                
                # Apply to all string columns at once
                for col in df_encoded.columns:
                    if df_encoded[col].dtype == 'object':  # Only process string columns
                        # Use vectorized string replacement
                        mask = df_encoded[col].astype(str).str.contains(
                            re.escape(tag), regex=True, na=False
                        )
                        if mask.any():
                            df_encoded.loc[mask, col] = df_encoded.loc[mask, col].astype(str).str.replace(
                                tag, encoded_tag, regex=False
                            )
        
        return df_encoded

### Step 5.3 Link Validator `<a>` tags

In [9]:
class LinkValidator:
    def __init__(self):
        # regex for raw <http://...> or <https://...>
        self.raw_link_pattern = re.compile(r'<(https?://[^>\s]+)>')
        # regex for opening <a> tags
        self.anchor_open_pattern = re.compile(r'<a([^>]*)>', flags=re.IGNORECASE)
        # regex for closing </a>
        self.anchor_close_pattern = re.compile(r'</a>', flags=re.IGNORECASE)
    
    def has_href_attribute(self, attrs_string):
        # Look for href attribute with optional whitespace around =
        href_pattern = r'href\s*=\s*["\'][^"\']*["\']'
        return bool(re.search(href_pattern, attrs_string, re.IGNORECASE))
   
    def validate_links(self, text: str) -> List[Dict]:
        """
        Validates links inside a text string.
        - Finds raw URLs in <> that should be <a href="">
        - Ensures <a> tags have href
        - Ensures <a> tags are properly closed
        """
        if pd.isna(text) or text == '':
            return []
        issues = []
        text_str = str(text)
        
        # 1. Detect raw <http://...>
        for url in self.raw_link_pattern.findall(text_str):
            issues.append({
                "Issue": "Raw URL in angle brackets",
                "URL": url,
                "Suggested_Fix": f'<a href="{url}">{url}</a>'
            })
        
        # 2. Detect <a> tags without href
        for attrs in self.anchor_open_pattern.findall(text_str):
            if not self.has_href_attribute(attrs):
                issues.append({
                    "Issue": "Anchor tag missing href",
                    "Tag": f"<a{attrs}>",
                    "Suggested_Fix": '<a href="URL_HERE">'
                })
        
        # 3. Check open/close balance
        open_count = len(self.anchor_open_pattern.findall(text_str))
        close_count = len(self.anchor_close_pattern.findall(text_str))
        if open_count != close_count:
            issues.append({
                "Issue": "Anchor tag mismatch",
                "Opens": open_count,
                "Closes": close_count,
                "Suggested_Fix": "Ensure each <a> has matching </a>"
            })
        
        return issues
    
    def repair_a_tags(self, df_columns: pd.DataFrame) -> pd.DataFrame:
        """
        Repair <a> tag issues in the dataframe columns.
        Currently handles:
        - Converting raw URLs in angle brackets to proper <a href=""> tags
        """
        repaired_df = df_columns.copy()
        repairs_made = 0
        
        for col in repaired_df.columns:
            for idx in repaired_df.index:
                value = repaired_df.at[idx, col]
                
                if pd.isna(value) or value == '':
                    continue
                    
                text_str = str(value)
                original_text = text_str
                
                # Fix raw URLs in angle brackets
                def replace_raw_url(match):
                    url = match.group(1)
                    return f'<a href="{url}">{url}</a>'
                
                text_str = self.raw_link_pattern.sub(replace_raw_url, text_str)
                
                # Update if changes were made
                if text_str != original_text:
                    repaired_df.at[idx, col] = text_str
                    repairs_made += 1
        
        print(f"  Repaired {repairs_made} raw URL issues")
        return repaired_df
    
    def analyze_link_issues(self, df_columns: pd.DataFrame) -> Dict:
        """
        Analyze all link issues in the dataframe columns.
        Returns structured analysis similar to other analyzers.
        """
        all_issues = []
        
        for col in df_columns.columns:
            for idx in df_columns.index:
                value = df_columns.at[idx, col]
                issues = self.validate_links(value)
                
                for issue in issues:
                    issue["Row"] = idx + 1  # Fix: Add 1 to match actual row numbers
                    issue["Column"] = col
                    all_issues.append(issue)
        
        issues_df = pd.DataFrame(all_issues) if all_issues else pd.DataFrame()
        
        return {
            'Link_Issues_by_Row': issues_df,
            'Total_Issues': len(all_issues),
            'Repairable_Issues': len([i for i in all_issues if i['Issue'] == 'Raw URL in angle brackets'])
        }

## Step 6 HTML Processor

In [10]:
class HTMLProcessor:
    """Main processor that coordinates all operations - OPTIMIZED"""
    
    def __init__(self):
        # Use YOUR existing comprehensive classes
        self.tag_handler = HTMLTagHandler()
        self.malformed_repairer = MalformedTagRepairer()
        self.non_html_processor = NonHTMLTagProcessor()
        self.link_validator = LinkValidator()
        self.stats = {}
        self.content_columns = [
            'Principal English Translation',
            'Attestations from sources in English',
            'Attestations from sources in Spanish',
            'Alonso de Molina',
            'Frances Karttunen', 
            'Horacio Carochi / English',
            'Andrés de Olmos',
            "Lockhart’s Nahuatl as Written",
            
        ]
    
    
    def  process_dataframe(
        self, df: pd.DataFrame, 
        repair_malformed: bool = True,
        validate_links: bool = True,
        extract_citations: bool = True,
        extract_crossrefs: bool = True,
        encode_non_html: bool = True
    ) -> Tuple[pd.DataFrame, Dict]:
        
        """
        Process entire dataframe efficiently, including:
        - Malformed HTML repair
        - Non-HTML tag encoding
        - <a> link validation
        Returns: (processed_df, analysis_report)
        """
        print("=" * 70)
        print("HTML PROCESSING PIPELINE")
        print("=" * 70)
        
        result_df = df.copy()
        
        # Add extraction columns if needed
        if extract_citations:
            result_df['Citations'] = ''
            result_df['Number_of_Citations'] = 0
        if extract_crossrefs:
            result_df['Cross_References'] = ''
            result_df['Number_of_Cross_References'] = 0
        
        analysis = {
            'repairs_made': 0,
            'citations_extracted': 0,
            'crossrefs_extracted': 0,
            'non_html_encoded': 0,
            'tag_analysis': [],
            'processing_details': []
        }
        report = {}
        reports = []  # collect all issues for unified mismatch report
        total_rows = len(result_df)
        print(f"\nProcessing {total_rows:,} rows...")

        # -------------------------------
        # STEP 1: Malformed HTML tags
        # -------------------------------
        if repair_malformed:
            print("  Analyzing malformed tags...")
            malformed_analysis = self.malformed_repairer.analyze_malformed_tags(result_df[self.content_columns])

            if not malformed_analysis['Malformed_Tags_Summary'].empty:
                stats = self.malformed_repairer.get_repair_statistics(malformed_analysis)
                print(f"  Found {stats['total_issues']} malformed tag instances")
                print(f"  Found {stats['unique_patterns']} unique malformed patterns")
                print(f"    - Auto-fixable: {stats['auto_fixable']}")
                print(f"    - Manual review needed: {stats['manual_review_needed']}")

                mismatch_report = self.malformed_repairer.generate_mismatch_report(malformed_analysis)
                if not mismatch_report.empty:
                    mismatch_report = mismatch_report.copy()
                    mismatch_report["Issue_Type"] = "Malformed Tag"
                    reports.append(mismatch_report)

                print("  Repairing malformed tags...")
                result_df[self.content_columns] = self.malformed_repairer.repair_tags(
                    result_df[self.content_columns],
                    malformed_analysis
                )

                analysis['repairs_made'] = stats['auto_fixable']
                analysis['bibl_repairs'] = stats.get('bibl_mismatches', 0)
                analysis['repair_stats'] = stats
                # optional raw malformed data
                analysis['malformed_repairs'] = malformed_analysis
            else:
                print("  No malformed tags found")

        # -------------------------------
        # STEP 2: Non-HTML tags
        # -------------------------------
        if encode_non_html:
            print("  Analyzing non-HTML tags...")
            non_html_analysis = self.non_html_processor.analyze_non_html_tags(result_df[self.content_columns])

            if not non_html_analysis['Non_HTML_Tags_Summary'].empty:
                print(f"  Found {len(non_html_analysis['Non_HTML_Tags_by_Row'])} non-HTML tag instances")
                print(f"  Found {len(non_html_analysis['Non_HTML_Tags_Summary'])} unique non-HTML tags")

                # Encode brackets in the dataframe
                result_df[self.content_columns] = self.non_html_processor.encode_brackets(
                    result_df[self.content_columns],
                    non_html_analysis
                )

                analysis['non_html_encoded'] = len(non_html_analysis['Non_HTML_Tags_by_Row'])
                analysis['non_html_encoding'] = non_html_analysis

                # Prepare report entries
                non_html_rows = []
                non_html_df = non_html_analysis['Non_HTML_Tags_by_Row']
                for _, row in non_html_df.iterrows():
                    non_html_rows.append({
                        "Row": row['Row'],
                        "Column": row['Column'],
                        "Issue": row['Non_HTML_Tag'],
                        "Issue_Type": "Non-HTML Tag"
                    })
                if non_html_rows:
                    reports.append(pd.DataFrame(non_html_rows))
            else:
                print("  No non-HTML tags found")

        # -------------------------------
        # STEP 3: Link validation
        # -------------------------------
        if validate_links:
            print("  Analyzing link issues...")
            link_analysis = self.link_validator.analyze_link_issues(result_df[self.content_columns])
            
            if link_analysis['Total_Issues'] > 0:
                print(f"  Found {link_analysis['Total_Issues']} link issues")
                print(f"  Repairable issues: {link_analysis['Repairable_Issues']}")
                
                # Repair what we can
                if link_analysis['Repairable_Issues'] > 0:
                    print("  Repairing link issues...")
                    result_df[self.content_columns] = self.link_validator.repair_a_tags(
                        result_df[self.content_columns]
                    )
                
                # Add remaining issues to report
                if not link_analysis['Link_Issues_by_Row'].empty:
                    link_report = link_analysis['Link_Issues_by_Row'].copy()
                    link_report["Issue_Type"] = "Link Issue"
                    reports.append(link_report)
                    
                analysis['link_repairs'] = link_analysis['Repairable_Issues']
                analysis['link_analysis'] = link_analysis
            else:
                print("  No link issues found")

        # -------------------------------
        # STEP 4: Unified mismatch report
        # -------------------------------
        if reports:
            unified_report = pd.concat(reports, ignore_index=True)
        else:
            unified_report = pd.DataFrame(columns=["Row", "Column", "Issue", "Issue_Type"])

        report['analysis'] = analysis
        report["mismatch_report"] = unified_report

        self._print_summary(analysis)
        return result_df, report

    def _print_summary(self, analysis: Dict):
        """Print processing summary"""
        print("\n" + "=" * 70)
        print("PROCESSING COMPLETE")
        print("=" * 70)
        print(f"Repairs made:        {analysis['repairs_made']:,}")
        print(f"Citations extracted: {analysis['citations_extracted']:,}")
        print(f"Cross-refs extracted: {analysis['crossrefs_extracted']:,}")
        print(f"Non-HTML encoded:    {analysis['non_html_encoded']:,}")

## Main Processor Class

In [11]:

class NahuatlProcessor:
    """Main processor class that coordinates all operations"""
    
    def __init__(self, db_path: str = DB_PATH, working_dir: str = WORKING_DIR):
        """Initialize the processor with all necessary components"""
        
        print("=" * 70)
        print("NAHUATL DATA PROCESSOR - HYBRID APPROACH")
        print("=" * 70)
        
        # Initialize components
        self.html_processor = HTMLProcessor()
        self.loader = DataLoader()
        self.saver = DataSaver(working_dir)
        self.db = DatabaseManager(db_path)
        
        # Data containers
        self.original_df = None
        self.working_df = None
        
        # Processing state
        self.current_stage = 'initialized'
        self.processing_history = []
        
        # Verify base dataset exists
        if not self.db.verify_base_dataset():
            print("\n No base dataset found in SQLite")
        print("   Run processor.initial_import() to import your CSV data first")
        print("Processor initialized and ready")
        print("=" * 70)
    
    def initial_import(self, csv_path: str, table_name: str = 'WHP_EarlyNahuatl_Data', 
                      encoding: str = DEFAULT_ENCODING, replace: bool = False):
        """
        Perform initial import of CSV data into SQLite.
        This establishes the base dataset in the database.
        """
        
        success = self.db.import_initial_dataset(
            csv_path=csv_path,
            table_name=table_name,
            encoding=encoding,
            replace=replace
        )
        
        if success:
            print("\nInitial import complete!")
            print("   You can now proceed with data processing")
        else:
            print("\nInitial import failed")
            
        return success
    
    def load_data(self, source: str, source_type: str = 'auto', **kwargs) -> pd.DataFrame:
        """Load data from any source"""
        
        start_time = datetime.now()
        
        # Auto-detect source type if needed
        if source_type == 'auto':
            source_type = 'csv' if source.endswith('.csv') else 'sqlite'
        
        # Load based on source type
        if source_type == 'csv':
            df = self.loader.load_from_csv(source, **kwargs)
        elif source_type == 'sqlite':
            table_name = kwargs.get('table_name', 'WHP_EarlyNahuatl_Data')
            df = self.loader.load_from_sqlite(source, table_name)
        else:
            raise ValueError(f"Unknown source type: {source_type}")
        
        # Create working copies
        self.original_df, self.working_df = self.loader.create_working_copy(df)
        
        # Save initial checkpoint
        self.save_checkpoint('initial')
        
        # Log the operation
        duration = (datetime.now() - start_time).total_seconds()
        self.db.log_processing_stage(
            stage='data_loading',
            status='completed',
            duration=duration,
            rows_processed=len(df),
            notes=f"Loaded from {source_type}: {source}"
        )
        
        self.current_stage = 'data_loaded'
        
        return self.working_df
    
    def save_checkpoint(
        self,
        checkpoint_name: str,
        additional_data: Optional[Dict[str, pd.DataFrame]] = None
    ):
        """Enhanced checkpoint saving with additional reports"""
        if self.working_df is not None and self.db.conn is not None:
            # Save main checkpoint
            self.saver.save_checkpoint_to_sqlite(
                self.working_df,
                checkpoint_name,
                self.db.conn
            )
            
            # Save additional data if provided (like mismatch reports)
            if additional_data:
                for data_name, data_df in additional_data.items():
                    if isinstance(data_df, pd.DataFrame) and not data_df.empty:
                        table_name = f'{data_name}_{checkpoint_name}_{datetime.now().strftime("%Y%m%d")}'
                        data_df.to_sql(
                            table_name,
                            self.db.conn,
                            if_exists='replace',
                            index=False
                        )
                        print(f"  Saved {data_name} to {table_name}")
            
            self.processing_history.append({
                'checkpoint': checkpoint_name,
                'timestamp': datetime.now(),
                'rows': len(self.working_df),
                'columns': len(self.working_df.columns)
            })
        else:
            print(f"Warning: Cannot save checkpoint '{checkpoint_name}' - no data or connection available")
    
    def export_final_results(self, filename_base: str = 'final_nahuatl_data'):
        """Export final results to CSV with encoding protection"""
        
        if self.working_df is None:
            print("Error: No working data to export")
            return
        
        if self.original_df is None:
            print("Warning: No original data for comparison")
            original_rows = 0
            original_cols = 0
        else:
            original_rows = len(self.original_df)
            original_cols = len(self.original_df.columns)
        
        # Save to CSV
        csv_path = self.saver.save_to_csv(self.working_df, f"{filename_base}.csv")
        
        # Save final checkpoint
        self.save_checkpoint('final')
        
        # Create summary report
        summary = {
            'processing_summary': pd.DataFrame(self.processing_history),
            'data_info': pd.DataFrame([{
                'original_rows': original_rows,
                'final_rows': len(self.working_df),
                'original_columns': original_cols,
                'final_columns': len(self.working_df.columns),
                'stages_completed': len(self.processing_history)
            }])
        }
        
        # Save summary to Excel
        excel_path = self.saver.save_to_excel(
            summary,
            f"{filename_base}_summary_{datetime.now().strftime('%Y%m%d')}.xlsx"
        )
        
        print("\n" + "=" * 70)
        print("EXPORT COMPLETE")
        print(f"CSV: {csv_path}")
        print(f"Summary: {excel_path}")
        print("=" * 70)
    
    def load_data_from_base(self, table_name: str = 'WHP_EarlyNahuatl_Data') -> pd.DataFrame:
        """Convenience method to load from the base SQLite dataset"""
        
        if not self.db.verify_base_dataset(table_name):
            raise ValueError(f"Base dataset '{table_name}' not found. Run initial_import() first.")
        
        return self.load_data(
            source=self.db.db_path,
            source_type='sqlite',
            table_name=table_name
        )
        
    def process_html_tags(self, save_checkpoint: bool = True):
        """Step 4-5: HTML tag processing"""
        if self.working_df is None:
            raise ValueError("No data loaded. Run load_data() first.")
        
        # Delegate to HTML processor
        self.working_df, report = self.html_processor.process_dataframe(
            self.working_df,
            repair_malformed=True,
            encode_non_html=True,
            validate_links=True,
        )
        
        if save_checkpoint:
            additional_data = {}
            
            # The HTMLProcessor already creates a unified mismatch_report
            # that includes ALL issues (malformed tags, non-HTML tags, link issues)
            # Each row already has an 'Issue_Type' column
            if 'mismatch_report' in report and not report['mismatch_report'].empty:
                additional_data['mismatch_report'] = report['mismatch_report']
                
                # Print summary by issue type
                print("\nIssue Summary:")
                issue_counts = report['mismatch_report']['Issue_Type'].value_counts()
                for issue_type, count in issue_counts.items():
                    print(f"  - {issue_type}: {count} issues")
                    
                print(f"\nTotal issues found: {len(report['mismatch_report'])}")
            else:
                print("\nNo issues found - data is clean!")
            
            # Save checkpoint with the unified report
            self.save_checkpoint('after_html_processing', additional_data)
            
        return report    

    def cleanup(self):
        """Clean up resources"""
        self.db.close()

At this point we've done enough thorough analysis and for the sake of correctness, manual correction will be done on the checkpoint table in the above log. After the corrections are made we will begin extracting citations and cross references.
Note: only after this stage since we did some manual cleaning up use this db:
checkpoint_final_20250914.db

now that we've extracted citations and cross references will be uploading the working version to be the checkpoint_after_citation_and_crossref_extraction_20250922 table.

this table will include the manual changes from the checkpoint_final_20250914 table.


## Step 7: Citation Extractor (Extract and Remove)

In [12]:
class CitationExtractor:
    """Extract and remove <bibl> citations from text"""
    
    def __init__(self):
        # Pattern for bibl tags with any attributes
        self.bibl_pattern = re.compile(
            r'<bibl[^>]*>(.*?)</bibl>',
            re.DOTALL | re.IGNORECASE
        )
        
    def extract_and_remove_citations(self, text: str) -> Tuple[str, List[str]]:
        """
        Extract citations from text and return cleaned text WITHOUT citations.
        Returns: (cleaned_text, list_of_citations)
        """
        if pd.isna(text) or text == '':
            return text, []
        
        text_str = str(text)
        citations = []
        
        # Find all bibl tags and their content
        matches = list(self.bibl_pattern.finditer(text_str))
        
        # Process in reverse order to preserve positions
        for match in reversed(matches):
            # Extract the ENTIRE bibl block (including the tags)
            full_citation = match.group(0)  # Changed from group(1) to group(0)
            
            # Clean up the citation content
            cleaned_citation = self._clean_citation_content(full_citation)
            
            if cleaned_citation:
                citations.insert(0, cleaned_citation)  # Insert at beginning to maintain order
                
            # Remove the entire bibl tag from original text
            start, end = match.span()
            text_str = text_str[:start] + text_str[end:]
        
        # Clean up any extra whitespace left behind
        text_str = re.sub(r'\s+', ' ', text_str).strip()
        
        return text_str, citations
    
    def _clean_citation_content(self, content: str) -> str:
        """Clean up extracted citation content"""
        if not content:
            return ""
        # normalize whitespace
        cleaned = content.strip()    
        # preserves content within tags while normalizing spaces between
        cleaned = re.sub(r'>\s+<', '><', cleaned)
        cleaned = re.sub(r'\s+',' ',cleaned)
        cleaned = cleaned.strip()
        if len(cleaned) > 0:
            return cleaned
        return ""
    
    def process_dataframe(self, df: pd.DataFrame, content_columns: List[str]) -> pd.DataFrame:
        """
        Process dataframe to extract and remove citations.
        Creates new citation columns and cleans original columns.
        """
        print("  Extracting and removing citations...")
        df_processed = df.copy()
        
        # Initialize new columns
        if 'Citations' not in df_processed.columns:
            df_processed['Citations'] = ''
        if 'Number_of_Citations' not in df_processed.columns:
            df_processed['Number_of_Citations'] = 0
            
        # Ensure numeric column is proper integer type
        df_processed['Number_of_Citations'] = df_processed['Number_of_Citations'].astype(int)
        
        total_citations_found = 0
        rows_with_citations = 0
        
        for idx, row in df_processed.iterrows():
            if idx % 5000 == 0:
                print(f"    Processing row {idx}...")
            
            all_citations = []
            
            # Process each content column
            for col in content_columns:
                if col not in df_processed.columns:
                    continue
                    
                cell_value = row[col]
                if pd.notna(cell_value) and str(cell_value).strip():
                    # Extract and remove citations
                    cleaned_text, citations = self.extract_and_remove_citations(cell_value)
                    
                    # Update the cell with cleaned text
                    df_processed.at[idx, col] = cleaned_text
                    
                    # Collect citations
                    all_citations.extend(citations)
            
            # Store citations in new columns
            if all_citations:
                df_processed.at[idx, 'Citations'] = ' | '.join(all_citations)
                df_processed.at[idx, 'Number_of_Citations'] = len(all_citations)
                total_citations_found += len(all_citations)
                rows_with_citations += 1
        
        print(f"    ✓ Extracted {total_citations_found} citations from {rows_with_citations} rows")
        print(f"    ✓ Removed <bibl> tags from original columns")
        print(f"    ✓ Preserved HTML structure within citations")
        
        return df_processed

## Step 8: Cross-Reference Extractor (Extract and Remove)

In [13]:
class CrossReferenceExtractor:
    """Extract and remove cross-references from text while preserving HTML structure"""
    
    def __init__(self):
        # Define cross-reference patterns to extract and remove
        # ORDER MATTERS: More specific patterns first to avoid duplicates
        self.crossref_patterns = [
            # Scholar references FIRST (most specific)
            (r'\(see\s+Molina[^)]*\)', 'molina_ref'),
            (r'\(see\s+Karttunen[^)]*\)', 'karttunen_ref'),
            (r'\(see\s+Carochi[^)]*\)', 'carochi_ref'),
            (r'\(see\s+Lockhart[^)]*\)', 'lockhart_ref'),
            
            # Then other parenthetical references
            (r'\(see\s+([^)]+)\)', 'see'),
            (r'\(cf\.\s+([^)]+)\)', 'cf'),
            (r'\(compare\s+([^)]+)\)', 'compare'),
            (r'\(also\s+see\s+([^)]+)\)', 'also_see'),
            (r'\(see\s+also\s+([^)]+)\)', 'see_also'),
            (r'\(variant\s+of\s+([^)]+)\)', 'variant'),
            (r'\(also\s+([^)]+)\)', 'also'),
            (r'\(See\s+([^)]+)\)', 'see'),  # Capital See
            
            # Bracketed references
            (r'\[see\s+([^\]]+)\]', 'see'),
            (r'\[cf\.\s+([^\]]+)\]', 'cf'),
        ]
        
        # Compile patterns for efficiency
        self.compiled_patterns = [
            (re.compile(pattern, re.IGNORECASE), ref_type) 
            for pattern, ref_type in self.crossref_patterns
        ]
    
    def extract_and_remove_crossrefs(self, text: str) -> Tuple[str, List[Dict[str, str]]]:
        """
        Extract cross-references and return cleaned text WITHOUT them.
        PRESERVES HTML tags in the remaining text.
        Returns: (cleaned_text, list_of_crossref_dicts)
        """
        if pd.isna(text) or text == '':
            return text, []
        
        text_str = str(text)
        crossrefs = []
        
        # Track positions we've already processed to avoid duplicates
        processed_ranges = []
        
        # Track all matches with their positions
        all_matches = []
        
        for pattern, ref_type in self.compiled_patterns:
            for match in pattern.finditer(text_str):
                start, end = match.span()
                
                # Check if this position overlaps with already processed text
                overlap = False
                for proc_start, proc_end in processed_ranges:
                    if (start >= proc_start and start < proc_end) or \
                       (end > proc_start and end <= proc_end):
                        overlap = True
                        break
                
                if overlap:
                    continue  # Skip this match, it's already been handled
                
                # Handle both patterns with and without capture groups
                if match.groups():  # Has capture groups
                    content = match.group(1)
                else:  # No capture groups - use the whole match
                    content = match.group(0)
                
                all_matches.append({
                    'match': match,
                    'type': ref_type,
                    'full_text': match.group(0),
                    'content': content
                })
                
                # Mark this range as processed
                processed_ranges.append((start, end))
        
        # Sort by position (reverse order for safe removal)
        all_matches.sort(key=lambda x: x['match'].start(), reverse=True)
        
        # Extract and remove each match
        for match_info in all_matches:
            match = match_info['match']
            
            # Clean the extracted content
            if match_info['type'] in ['molina_ref', 'karttunen_ref', 'carochi_ref', 'lockhart_ref']:
                # For scholar refs, clean the full match
                cleaned_content = match_info['full_text'].strip('()[]')
            else:
                # For other refs, clean the captured group
                cleaned_content = self._clean_crossref_content(match_info['content'])
            
            if cleaned_content:
                crossrefs.insert(0, {
                    'text': cleaned_content,
                    'type': match_info['type']
                })
                
            # Remove from original text
            start, end = match.span()
            
            # Also remove a preceding space if there is one
            if start > 0 and text_str[start-1] == ' ':
                text_str = text_str[:start-1] + text_str[end:]
            else:
                text_str = text_str[:start] + text_str[end:]
        
        # Minimal cleanup - just normalize excessive whitespace
        text_str = re.sub(r'  +', ' ', text_str)
        
        return text_str, crossrefs
    
    def _clean_crossref_content(self, content: str) -> str:
        """Clean up extracted cross-reference content"""
        if not content:
            return ""
        
        # Remove any HTML tags from the cross-reference content
        cleaned = re.sub(r'<[^>]+>', '', content)
        
        # Normalize whitespace
        cleaned = ' '.join(cleaned.strip().split())
        
        # Remove trailing punctuation
        cleaned = cleaned.strip('.,;:')
        
        # Must be substantial
        if len(cleaned) > 1 and any(c.isalnum() for c in cleaned):
            return cleaned.strip()
        
        return ""
    
    def process_dataframe(self, df: pd.DataFrame, content_columns: List[str]) -> pd.DataFrame:
        """
        Process dataframe to extract and remove cross-references.
        Creates new cross-reference columns and cleans original columns.
        """
        print("  Extracting and removing cross-references...")
        print("  NOTE: Preserving HTML tag structure")
        df_processed = df.copy()
        
        # Initialize new columns if they don't exist
        if 'Cross_References' not in df_processed.columns:
            df_processed['Cross_References'] = ''
        if 'Number_of_Cross_References' not in df_processed.columns:
            df_processed['Number_of_Cross_References'] = 0
        if 'CrossRef_Types' not in df_processed.columns:
            df_processed['CrossRef_Types'] = ''
        
        # Ensure numeric column is proper type
        df_processed['Number_of_Cross_References'] = pd.to_numeric(
            df_processed['Number_of_Cross_References'], 
            errors='coerce'
        ).fillna(0).astype(int)
        
        total_crossrefs_found = 0
        rows_with_crossrefs = 0
        
        for idx, row in df_processed.iterrows():
            if idx % 5000 == 0:
                print(f"    Processing row {idx}...")
            
            all_crossrefs = []
            all_types = set()
            
            # Process each content column
            for col in content_columns:
                if col not in df_processed.columns:
                    continue
                # Skip our newly added columns
                if col in ['Citations', 'Number_of_Citations', 'Cross_References', 
                          'Number_of_Cross_References', 'CrossRef_Types']:
                    continue
                    
                cell_value = row[col]
                if pd.notna(cell_value) and str(cell_value).strip():
                    try:
                        # Extract and remove cross-references
                        cleaned_text, crossrefs = self.extract_and_remove_crossrefs(cell_value)
                        
                        # Update the cell with cleaned text
                        df_processed.at[idx, col] = cleaned_text
                        
                        # Collect cross-references
                        for crossref in crossrefs:
                            all_crossrefs.append(crossref['text'])
                            all_types.add(crossref['type'])
                    except Exception as e:
                        print(f"    Warning: Error processing row {idx}, column {col}: {e}")
                        continue
            
            # Store cross-references in new columns
            if all_crossrefs:
                df_processed.at[idx, 'Cross_References'] = ' | '.join(all_crossrefs)
                df_processed.at[idx, 'Number_of_Cross_References'] = len(all_crossrefs)
                df_processed.at[idx, 'CrossRef_Types'] = ', '.join(sorted(all_types))
                total_crossrefs_found += len(all_crossrefs)
                rows_with_crossrefs += 1
        
        print(f"    ✓ Extracted {total_crossrefs_found} cross-references from {rows_with_crossrefs} rows")
        print(f"    ✓ Removed cross-reference patterns from original columns")
        print(f"    ✓ Preserved HTML structure in remaining content")
        
        return df_processed

In [14]:
def extract_citations_and_crossrefs(processor: NahuatlProcessor, 
                                    save_checkpoint: bool = True,
                                    content_columns: Optional[List[str]] = None) -> Dict:
    """
    Extract citations and cross-references from the processor's working dataframe.
    
    Args:
        processor: NahuatlProcessor instance with loaded data
        save_checkpoint: Whether to save checkpoint after both extractions
        content_columns: List of columns to process (uses default if None)
    
    Returns:
        Dictionary with extraction statistics
    """
    if processor.working_df is None:
        raise ValueError("No data loaded in processor. Run processor.load_data() first.")
    
    print("\n" + "=" * 70)
    print("CITATION AND CROSS-REFERENCE EXTRACTION")
    print("=" * 70)
    
    # Use default content columns if not provided
    if content_columns is None:
        content_columns = [
            'Principal English Translation',
            'Attestations from sources in English',
            'Attestations from sources in Spanish',
            'Alonso de Molina',
            'Frances Karttunen',
            'Horacio Carochi / English',
            'Andrés de Olmos',
            "Lockhart’s Nahuatl as Written",

        ]
    
    # Step 1: Extract and remove citations
    print("\nStep 1: Citation Extraction")
    print("-" * 40)
    citation_extractor = CitationExtractor()
    processor.working_df = citation_extractor.process_dataframe(
        processor.working_df,
        content_columns
    )
    
    # Step 2: Extract and remove cross-references
    print("\nStep 2: Cross-Reference Extraction")
    print("-" * 40)
    crossref_extractor = CrossReferenceExtractor()
    processor.working_df = crossref_extractor.process_dataframe(
        processor.working_df,
        content_columns
    )
    
    processor.working_df['Number_of_Citations'] = pd.to_numeric(
        processor.working_df['Number_of_Citations'], errors='coerce'
    ).fillna(0).astype(int)
    
    processor.working_df['Number_of_Cross_References'] = pd.to_numeric(
        processor.working_df['Number_of_Cross_References'], errors='coerce'
    ).fillna(0).astype(int)
    
    # Save single combined checkpoint after both extractions are complete
    if save_checkpoint:
        processor.save_checkpoint('after_citation_and_crossref_extraction')
    
    # Generate summary statistics
    stats = {
        'total_rows': len(processor.working_df),
        'rows_with_citations': len(processor.working_df[processor.working_df['Number_of_Citations'] > 0]),
        'total_citations': processor.working_df['Number_of_Citations'].sum(),
        'max_citations_per_row': processor.working_df['Number_of_Citations'].max(),
        'avg_citations_per_row': processor.working_df['Number_of_Citations'].mean(),
        'rows_with_crossrefs': len(processor.working_df[processor.working_df['Number_of_Cross_References'] > 0]),
        'total_crossrefs': processor.working_df['Number_of_Cross_References'].sum(),
        'max_crossrefs_per_row': processor.working_df['Number_of_Cross_References'].max(),
        'avg_crossrefs_per_row': processor.working_df['Number_of_Cross_References'].mean(),
    }
    
    # Print summary
    print("\n" + "=" * 70)
    print("EXTRACTION SUMMARY")
    print("=" * 70)
    print(f"Total rows processed: {stats['total_rows']:,}")
    print("\nCitations:")
    print(f"  - Rows with citations: {stats['rows_with_citations']:,} ({stats['rows_with_citations']/stats['total_rows']*100:.1f}%)")
    print(f"  - Total citations extracted: {stats['total_citations']:,}")
    print(f"  - Average per row: {stats['avg_citations_per_row']:.2f}")
    print(f"  - Maximum per row: {stats['max_citations_per_row']}")
    print("\nCross-References:")
    print(f"  - Rows with cross-references: {stats['rows_with_crossrefs']:,} ({stats['rows_with_crossrefs']/stats['total_rows']*100:.1f}%)")
    print(f"  - Total cross-references extracted: {stats['total_crossrefs']:,}")
    print(f"  - Average per row: {stats['avg_crossrefs_per_row']:.2f}")
    print(f"  - Maximum per row: {stats['max_crossrefs_per_row']}")
    print("=" * 70)
    
    return stats

extracted citations and cross references into this checkpoint: checkpoint_after_citation_and_crossref_extraction_2025092
there was an issue regarding there being empty "p" tags in the Principal English Translation column

In [ ]:
def remove_empty_p_tags(text):
    """
    Remove empty <p></p> tags from text, including those with only whitespace
    """
    # Pattern to match <p></p> tags with optional whitespace inside and around
    pattern = r'\s*<p>\s*</p>\s*'
    
    # Replace all matches with empty string
    cleaned_text = re.sub(pattern, '', text)
    
    # Clean up any multiple spaces that might be left
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()
    
    return cleaned_text

# Example usage:

# If you have a pandas DataFrame column: = 
processor = NahuatlProcessor()
processor.load_data("../../data/sqLiteDb/nahuatl_processing.db", source_type='sqlite', table_name='checkpoint_after_citation_and_crossref_extraction_20250922')

# processor.working_df['Principal English Translation'] = processor.working_df['Principal English Translation'].apply(remove_empty_p_tags)

NAHUATL DATA PROCESSOR - HYBRID APPROACH
Database initialized: ../../data/sqLiteDb/nahuatl_processing.db
Existing database objects: 10
   - checkpoint_metadata (table): 77 rows
   - sqlite_sequence (table): 2 rows
   - processing_log (table): 39 rows
   - WHP_EarlyNahuatl_Data (table): 31,806 rows
   - import_history (table): 1 rows
   - checkpoint_after_citation_and_crossref_extraction_20250922 (table): 31,806 rows
   - checkpoint_initial_20250922 (table): 31,806 rows
   - checkpoint_initial_20250929 (table): 31,806 rows
   - checkpoint_removed_empty_p_tags_20250929 (table): 31,806 rows
   - checkpoint_initial_20251002 (table): 31,806 rows
   Base dataset verified:
   Table: WHP_EarlyNahuatl_Data
   Rows: 31,806
   Columns: 13
   Run processor.initial_import() to import your CSV data first
Processor initialized and ready
Loading data from SQLite: ../../data/sqLiteDb/nahuatl_processing.db
Table: checkpoint_after_citation_and_crossref_extraction_20250922
Successfully loaded 31,806 rows 

In [18]:
processor.saver.save_checkpoint_to_sqlite(processor.working_df, "removed_empty_p_tags", processor.db.conn)

Checkpoint saved to SQLite: checkpoint_removed_empty_p_tags_20251002


In [20]:
def extract_lockhart_citations(processor: NahuatlProcessor) -> Dict:
    """
    Extract citations from Lockhart's column and append to existing Citations.
    Handles the apostrophe character issue and updates citation counts.
    """
    print("\n" + "=" * 70)
    print("LOCKHART COLUMN CITATION EXTRACTION (補充)")
    print("=" * 70)
    
    # Find the actual column name with the special apostrophe
    lockhart_col = None
    for col in processor.working_df.columns:
        if 'Lockhart' in col and 'Nahuatl' in col:
            lockhart_col = col
            break
    
    if lockhart_col is None:
        print(" Lockhart column not found!")
        return {'error': 'Column not found'}
    
    print(f"✓ Found column: '{lockhart_col}'")
    
    # Initialize the citation extractor
    citation_extractor = CitationExtractor()
    
    # Track statistics
    rows_updated = 0
    new_citations_found = 0
    
    # Process each row
    for idx, row in processor.working_df.iterrows():
        if idx % 5000 == 0:
            print(f"  Processing row {idx}...")
        
        lockhart_content = row[lockhart_col]
        
        # Skip empty cells
        if pd.isna(lockhart_content) or not str(lockhart_content).strip():
            continue
        
        # Extract citations from Lockhart column
        cleaned_text, new_citations = citation_extractor.extract_and_remove_citations(lockhart_content)
        
        if new_citations:
            # Update the Lockhart column with cleaned text
            processor.working_df.at[idx, lockhart_col] = cleaned_text
            
            # Get existing citations
            existing_citations = row.get('Citations', '')
            existing_count = row.get('Number_of_Citations', 0)
            
            # Append new citations
            if existing_citations and str(existing_citations).strip():
                # Merge with existing citations using | delimiter
                combined_citations = f"{existing_citations} | {' | '.join(new_citations)}"
            else:
                # No existing citations, just use new ones
                combined_citations = ' | '.join(new_citations)
            
            # Update dataframe
            processor.working_df.at[idx, 'Citations'] = combined_citations
            processor.working_df.at[idx, 'Number_of_Citations'] = str(existing_count) + str(len(new_citations))
            
            rows_updated += 1
            new_citations_found += len(new_citations)
    
    # Ensure citation count is integer type
    processor.working_df['Number_of_Citations'] = pd.to_numeric(
        processor.working_df['Number_of_Citations'], 
        errors='coerce'
    ).fillna(0).astype(int)
    
    # Print summary
    print("\n" + "=" * 70)
    print("LOCKHART EXTRACTION SUMMARY")
    print("=" * 70)
    print(f"Column processed: '{lockhart_col}'")
    print(f"Rows updated: {rows_updated:,}")
    print(f"New citations extracted: {new_citations_found:,}")
    print(f"Average citations per updated row: {new_citations_found/rows_updated:.2f}" if rows_updated > 0 else "N/A")
    print("=" * 70)
    
    return {
        'column_name': lockhart_col,
        'rows_updated': rows_updated,
        'new_citations_found': new_citations_found,
        'success': True
    }


# Usage:
results = extract_lockhart_citations(processor)

# Optional: Save checkpoint after extraction
if results.get('success'):
    processor.save_checkpoint('after_lockhart_citation_extraction')
    
    # Optionally save to CSV to verify
    # processor.saver.save_to_csv(
    #     processor.working_df, 
    #     "lockhart_citations_updated"
    # )


LOCKHART COLUMN CITATION EXTRACTION (補充)
✓ Found column: 'Lockhart’s Nahuatl as Written'
  Processing row 0...
  Processing row 5000...
  Processing row 10000...
  Processing row 15000...
  Processing row 20000...
  Processing row 25000...
  Processing row 30000...

LOCKHART EXTRACTION SUMMARY
Column processed: 'Lockhart’s Nahuatl as Written'
Rows updated: 907
New citations extracted: 924
Average citations per updated row: 1.02
Checkpoint saved to SQLite: checkpoint_after_lockhart_citation_extraction_20251002


In [21]:
def remove_empty_p_tags(text):
    """
    Remove empty <p></p> tags from text, including those with only whitespace
    """
    if pd.isna(text) or text == '':
        return text
    
    # Pattern to match <p></p> tags with optional whitespace inside and around
    pattern = r'\s*<p>\s*</p>\s*'
    
    # Replace all matches with empty string
    cleaned_text = re.sub(pattern, '', str(text))
    
    # Clean up any multiple spaces that might be left
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()
    
    return cleaned_text


def clean_empty_p_tags_from_columns(processor: NahuatlProcessor, 
                                     content_columns: Optional[List[str]] = None) -> Dict:
    """
    Remove empty <p></p> tags from all content columns.
    Handles missing columns and special characters gracefully.
    """
    print("\n" + "=" * 70)
    print("REMOVING EMPTY <p></p> TAGS FROM CONTENT COLUMNS")
    print("=" * 70)
    
    # Default content columns if not provided
    if content_columns is None:
        content_columns = [
            'Principal English Translation',
            'Attestations from sources in English',
            'Attestations from sources in Spanish',
            'Alonso de Molina',
            'Frances Karttunen',
            'Horacio Carochi / English',
            'Andrés de Olmos',
            "Lockhart’s Nahuatl as Written",
        ]
    
    # Track what we process
    columns_processed = []
    columns_not_found = []
    total_changes = 0
    
    # Process each column
    for target_col in content_columns:
        # Find the actual column (handles apostrophe variations)
        matched_col = None
        
        # Try exact match first
        if target_col in processor.working_df.columns:
            matched_col = target_col
        else:
            # Try fuzzy match (for apostrophe issues)
            for col in processor.working_df.columns:
                # Remove special characters for comparison
                normalized_target = target_col.replace("'", "").replace("'", "").lower()
                normalized_col = col.replace("'", "").replace("'", "").lower()
                if normalized_target == normalized_col:
                    matched_col = col
                    break
        
        if matched_col:
            print(f"Processing: '{matched_col}'")
            
            # Count changes
            before = processor.working_df[matched_col].astype(str)
            processor.working_df[matched_col] = processor.working_df[matched_col].apply(remove_empty_p_tags)
            after = processor.working_df[matched_col].astype(str)
            
            changes = (before != after).sum()
            total_changes += changes
            
            columns_processed.append(matched_col)
            print(f"  ✓ Modified {changes:,} cells")
        else:
            columns_not_found.append(target_col)
            print(f"  ⚠ Column not found: '{target_col}'")
    
    # Summary
    print("\n" + "=" * 70)
    print("CLEANING SUMMARY")
    print("=" * 70)
    print(f"Columns processed: {len(columns_processed)}/{len(content_columns)}")
    print(f"Total cells modified: {total_changes:,}")
    
    if columns_not_found:
        print(f"\nColumns not found ({len(columns_not_found)}):")
        for col in columns_not_found:
            print(f"  - {col}")
    
    print("=" * 70)
    
    return {
        'columns_processed': columns_processed,
        'columns_not_found': columns_not_found,
        'total_changes': total_changes
    }


# Usage:
processor = NahuatlProcessor()
processor.load_data(
    "../../data/sqLiteDb/nahuatl_processing.db", 
    source_type='sqlite', 
    table_name='checkpoint_after_lockhart_citation_extraction_20251002'
)

# Clean all content columns
results = clean_empty_p_tags_from_columns(processor)

# Save checkpoint
processor.save_checkpoint('after_empty_p_tag_removal')

# Optional: Save to CSV
processor.saver.save_to_csv(processor.working_df, "empty_p_tags_removed")

NAHUATL DATA PROCESSOR - HYBRID APPROACH
Database initialized: ../../data/sqLiteDb/nahuatl_processing.db
Existing database objects: 12
   - checkpoint_metadata (table): 80 rows
   - sqlite_sequence (table): 2 rows
   - processing_log (table): 40 rows
   - WHP_EarlyNahuatl_Data (table): 31,806 rows
   - import_history (table): 1 rows
   - checkpoint_after_citation_and_crossref_extraction_20250922 (table): 31,806 rows
   - checkpoint_initial_20250922 (table): 31,806 rows
   - checkpoint_initial_20250929 (table): 31,806 rows
   - checkpoint_removed_empty_p_tags_20250929 (table): 31,806 rows
   - checkpoint_initial_20251002 (table): 31,806 rows
   - checkpoint_removed_empty_p_tags_20251002 (table): 31,806 rows
   - checkpoint_after_lockhart_citation_extraction_20251002 (table): 31,806 rows
   Base dataset verified:
   Table: WHP_EarlyNahuatl_Data
   Rows: 31,806
   Columns: 13
   Run processor.initial_import() to import your CSV data first
Processor initialized and ready
Loading data from 

'working_files\\empty_p_tags_removed_20251002_142358.csv'

In [13]:
def create_batched_exports(
    processor: NahuatlProcessor,
    batch_size: int = 1000,
    create_plan_b: bool = True,
    plan_b_test_size: int = 10,
    output_prefix: str = "nahuatl_batch"
) -> Dict:
    """
    Split dataset into batches and create Plan B test files.
    
    Args:
        processor: NahuatlProcessor with loaded data
        batch_size: Rows per batch (default 1000)
        create_plan_b: Whether to create Plan B split of first batch
        plan_b_test_size: Size of Plan B test file (default 10)
        output_prefix: Prefix for output filenames
    
    Returns:
        Dictionary with file paths and statistics
    """
    if processor.working_df is None:
        raise ValueError("No data loaded in processor")
    
    print("\n" + "=" * 70)
    print("CREATING BATCHED EXPORTS")
    print("=" * 70)
    
    df = processor.working_df
    total_rows = len(df)
    num_batches = (total_rows + batch_size - 1) // batch_size  # Ceiling division
    
    print(f"Total rows: {total_rows:,}")
    print(f"Batch size: {batch_size}")
    print(f"Number of batches: {num_batches}")
    
    # Prepare output directory
    output_dir = Path(processor.saver.working_dir)
    batch_dir = output_dir / "batched_output"
    batch_dir.mkdir(parents=True, exist_ok=True)
    
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    
    created_files = {
        'regular_batches': [],
        'plan_b_files': [],
        'summary_file': None
    }
    
    # === PLAN B: First batch split ===
    if create_plan_b:
        print("\n" + "-" * 70)
        print("CREATING PLAN B FILES (Test on small batch first)")
        print("-" * 70)
        
        first_batch = df.iloc[:batch_size].copy()
        
        # Plan B Test file (first 10 rows)
        test_df = first_batch.iloc[:plan_b_test_size]
        test_filename = f"{output_prefix}_PLAN_B_TEST_{plan_b_test_size}rows_{timestamp}.csv"
        test_path = batch_dir / test_filename
        
        test_df.to_csv(
            test_path,
            index=False,
            encoding=processor.saver.encoding,
            na_rep='',
            quoting=csv.QUOTE_MINIMAL
        )
        created_files['plan_b_files'].append({
            'path': str(test_path),
            'rows': len(test_df),
            'description': f'Plan B TEST - First {plan_b_test_size} rows'
        })
        print(f"✓ Created TEST file: {test_filename}")
        print(f"  Rows: {len(test_df)}")
        
        # Plan B Remaining file (remaining 990 rows)
        remaining_df = first_batch.iloc[plan_b_test_size:]
        remaining_filename = f"{output_prefix}_PLAN_B_REMAINING_{len(remaining_df)}rows_{timestamp}.csv"
        remaining_path = batch_dir / remaining_filename
        
        remaining_df.to_csv(
            remaining_path,
            index=False,
            encoding=processor.saver.encoding,
            na_rep='',
            quoting=csv.QUOTE_MINIMAL
        )
        created_files['plan_b_files'].append({
            'path': str(remaining_path),
            'rows': len(remaining_df),
            'description': f'Plan B REMAINING - Rows {plan_b_test_size + 1} to {batch_size}'
        })
        print(f"✓ Created REMAINING file: {remaining_filename}")
        print(f"  Rows: {len(remaining_df)}")
    
    # === REGULAR BATCHES ===
    print("\n" + "-" * 70)
    print("CREATING REGULAR BATCHES")
    print("-" * 70)
    
    for batch_num in range(num_batches):
        start_idx = batch_num * batch_size
        end_idx = min(start_idx + batch_size, total_rows)
        
        batch_df = df.iloc[start_idx:end_idx].copy()
        
        # Create filename with batch info
        filename = f"{output_prefix}_{batch_num + 1:02d}_of_{num_batches:02d}_rows_{start_idx + 1}-{end_idx}_{timestamp}.csv"
        filepath = batch_dir / filename
        
        # Save batch
        batch_df.to_csv(
            filepath,
            index=False,
            encoding=processor.saver.encoding,
            na_rep='',
            quoting=csv.QUOTE_MINIMAL
        )
        
        created_files['regular_batches'].append({
            'batch_number': batch_num + 1,
            'path': str(filepath),
            'rows': len(batch_df),
            'row_range': f"{start_idx + 1}-{end_idx}"
        })
        
        print(f"✓ Batch {batch_num + 1:2d}/{num_batches}: {filename}")
        print(f"  Rows: {len(batch_df)} (rows {start_idx + 1}-{end_idx})")
    
    # === CREATE SUMMARY FILE ===
    print("\n" + "-" * 70)
    print("CREATING SUMMARY FILE")
    print("-" * 70)
    
    summary_data = {
        'Batch Overview': [],
        'Plan B Files': [],
        'Processing Instructions': []
    }
    
    # Batch overview
    for batch_info in created_files['regular_batches']:
        summary_data['Batch Overview'].append({
            'Batch': batch_info['batch_number'],
            'Filename': Path(batch_info['path']).name,
            'Rows': batch_info['rows'],
            'Row Range': batch_info['row_range']
        })
    
    # Plan B files
    if create_plan_b:
        for plan_b_info in created_files['plan_b_files']:
            summary_data['Plan B Files'].append({
                'Filename': Path(plan_b_info['path']).name,
                'Rows': plan_b_info['rows'],
                'Description': plan_b_info['description']
            })
    
    # Instructions
    summary_data['Processing Instructions'].append({
        'Step': 1,
        'Action': f'Test with Plan B TEST file ({plan_b_test_size} rows)',
        'File': Path(created_files['plan_b_files'][0]['path']).name if create_plan_b else 'N/A'
    })
    summary_data['Processing Instructions'].append({
        'Step': 2,
        'Action': 'If successful, add Plan B REMAINING file (990 rows)',
        'File': Path(created_files['plan_b_files'][1]['path']).name if create_plan_b else 'N/A'
    })
    summary_data['Processing Instructions'].append({
        'Step': 3,
        'Action': 'If successful, process remaining batches in order',
        'File': 'Batches 02 through ' + f"{num_batches:02d}"
    })
    
    # Save summary to Excel
    summary_filename = f"{output_prefix}_SUMMARY_{timestamp}.xlsx"
    summary_path = batch_dir / summary_filename
    
    with pd.ExcelWriter(summary_path, engine='openpyxl') as writer:
        for sheet_name, data in summary_data.items():
            pd.DataFrame(data).to_excel(writer, sheet_name=sheet_name, index=False)
    
    created_files['summary_file'] = str(summary_path)
    print(f"✓ Created summary: {summary_filename}")
    
    # === FINAL SUMMARY ===
    print("\n" + "=" * 70)
    print("BATCH EXPORT COMPLETE")
    print("=" * 70)
    print(f"Output directory: {batch_dir}")
    print(f"\nFiles created:")
    if create_plan_b:
        print(f"  Plan B files: {len(created_files['plan_b_files'])}")
    print(f"  Regular batches: {len(created_files['regular_batches'])}")
    print(f"  Summary file: 1")
    print(f"\nTotal data rows exported: {total_rows:,}")
    print("\nRECOMMENDED PROCESSING ORDER:")
    print("  1. Start with PLAN_B_TEST file (10 rows)")
    print("  2. If successful, add PLAN_B_REMAINING file (990 rows)")
    print("  3. If successful, process batches 02 through " + f"{num_batches:02d}")
    print("=" * 70)
    
    return created_files


# Usage:
processor = NahuatlProcessor()

# Load the combined CSV we just created
processor.load_data(
    "../scripts/export_to_csv_prototype/data/nahuatl_batch_import.csv",  # The file we just created
    source_type='csv'
)

# Create batched files with Plan B
files = create_batched_exports(
    processor,
    batch_size=1000,
    create_plan_b=True,
    plan_b_test_size=10,
    output_prefix="nahuatl_batch"
)

# Print file locations for easy access
print("\n FILE LOCATIONS:")
print("\nPlan B Files:")
for file_info in files['plan_b_files']:
    print(f"  {Path(file_info['path']).name}")

print("\nRegular Batches:")
for file_info in files['regular_batches'][:3]:  # Show first 3
    print(f"  {Path(file_info['path']).name}")
if len(files['regular_batches']) > 3:
    print(f"  ... and {len(files['regular_batches']) - 3} more batches")

print(f"\nSummary: {Path(files['summary_file']).name}")

NAHUATL DATA PROCESSOR - HYBRID APPROACH
Database initialized: ../../data/sqLiteDb/nahuatl_processing.db
Existing database objects: 14
   - checkpoint_metadata (table): 83 rows
   - sqlite_sequence (table): 2 rows
   - processing_log (table): 42 rows
   - WHP_EarlyNahuatl_Data (table): 31,806 rows
   - import_history (table): 1 rows
   - checkpoint_after_citation_and_crossref_extraction_20250922 (table): 31,806 rows
   - checkpoint_initial_20250922 (table): 31,806 rows
   - checkpoint_initial_20250929 (table): 31,806 rows
   - checkpoint_removed_empty_p_tags_20250929 (table): 31,806 rows
   - checkpoint_removed_empty_p_tags_20251002 (table): 31,806 rows
   - checkpoint_after_lockhart_citation_extraction_20251002 (table): 31,806 rows
   - checkpoint_after_empty_p_tag_removal_20251002 (table): 31,806 rows
   - checkpoint_initial_20251002 (table): 31,806 rows
   - IDIEZ_modern_nahuatl-all-2024-03-27T09-45-31 (table): 6,846 rows
   Base dataset verified:
   Table: WHP_EarlyNahuatl_Data
   